In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import preprocessing

In [10]:
df = pd.read_csv("Huston College Lawsuit/Lawsuit.csv")

In [11]:
df.head()

,Dept,Gender,Clin,Cert,Prate,Exper,Rank,Sal94,Sal95
0,1,1,0,0,7.4,9,3,77836,84612
1,1,1,0,0,6.7,10,2,69994,78497
2,1,1,0,0,8.1,6,1,62872,67756
3,1,1,1,1,5.1,27,3,155196,173220
4,1,1,0,0,7.0,10,3,89268,96099


In [19]:
df.describe()

,Prate,Exper,Sal94,Sal95,Dept_1,Dept_2,Dept_3,Dept_4,Dept_5,Dept_6,Gender_0,Gender_1,Clin_0,Clin_1,Cert_0,Cert_1,Rank_1,Rank_2,Rank_3
count,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000
mean,4.932184,10.229885,153593.344828,168906.655172,0.191571,0.153257,0.080460,0.114943,0.306513,0.153257,0.406130,0.593870,0.386973,0.613027,0.279693,0.720307,0.429119,0.245211,0.325670
std,1.944430,6.227176,80469.666720,88778.425411,0.394293,0.360927,0.272526,0.319565,0.461931,0.360927,0.492053,0.492053,0.487993,0.487993,0.449711,0.449711,0.495901,0.431039,0.469525
min,1.300000,1.000000,34514.000000,38675.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,6.000000,90771.000000,99972.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.400000,9.000000,133284.000000,148117.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,6.900000,14.000000,200543.000000,218955.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,8.700000,37.000000,428876.000000,472589.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
dff = df["Sal95"] - df["Sal94"]

In [22]:
dff.describe()

count      261.000000
mean     15313.310345
std       8926.154373
min       2312.000000
25%       8494.000000
50%      12955.000000
75%      19364.000000
max      53460.000000
dtype: float64

In [13]:
categorical = ["Dept","Gender","Clin","Cert","Rank"]

In [12]:
categorical = df.select_dtypes(include=['object']).columns.tolist()
categorical

[]

In [14]:
#Deep copy the original data
data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)

Feature Dept
mapping {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}
Feature Gender
mapping {0: 0, 1: 1}
Feature Clin
mapping {0: 0, 1: 1}
Feature Cert
mapping {0: 0, 1: 1}
Feature Rank
mapping {1: 0, 2: 1, 3: 2}


In [15]:
# Create new dataframe with dummy features
categorical_feats = df.select_dtypes(include=['object']).columns.tolist()
print(categorical_feats)

[]


In [16]:
df = pd.get_dummies(df, columns= categorical_feats)

In [17]:
df.head()

,Prate,Exper,Sal94,Sal95,Dept_1,Dept_2,Dept_3,Dept_4,Dept_5,Dept_6,Gender_0,Gender_1,Clin_0,Clin_1,Cert_0,Cert_1,Rank_1,Rank_2,Rank_3
0,7.4,9,77836,84612,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1
1,6.7,10,69994,78497,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0
2,8.1,6,62872,67756,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0
3,5.1,27,155196,173220,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1
4,7.0,10,89268,96099,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1


In [18]:
df.to_csv("LawsuitBinary",index=None,sep=";")